In [ ]:
!nvidia-smi
# 64 batchsize 11441MiB cant use
# 64 batchsize 15079Mib can

Wed Aug 19 02:09:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install biobert-embedding==0.1.2

     |████████████████████████████████| 778kB 5.4MB/s 
     |████████████████████████████████| 3.0MB 39.4MB/s 
     |████████████████████████████████| 1.1MB 30.8MB/s 
     |████████████████████████████████| 890kB 12.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=acf5298e8a1d9126cf141a48b69b056026b7dcc402b6547700aa372c8340d119
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 748.8MB 1.2MB/s eta 0:00:01

In [ ]:
import numpy as np
import xml.etree.ElementTree as ET
import os
import torch
from torch.utils.data import TensorDataset, random_split
from biobert_embedding.embedding import BiobertEmbedding
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel
import easydict
import random
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
biobert = BiobertEmbedding()
tokenizer = biobert.tokenizer

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

In [ ]:
def help_mask(text, tokenizer, max_length, sequence_a_segment_id = 0):
    tokens = tokenizer.tokenize(text)
    output = dict()
    special_tokens_count = 1  # only CLS token added
    if len(tokens) > max_length - special_tokens_count:
        tokens = tokens[:(max_length - special_tokens_count)]
    e1 = []
    e2 = []
    for i in range(len(tokens)-2):
        if tokens[i] == 'drug' and tokens[i+1] == '##1' and tokens[i+2] == "#":
            e1.append(i)
        if tokens[i] == 'drug' and tokens[i+1] == '##2' and tokens[i+2] == "$":
            e2.append(i)
    tokens = ['[CLS]'] + tokens
    token_type_ids = [sequence_a_segment_id] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    attention_mask = [1] * len(input_ids)
    padding_length = max_length - len(input_ids)
    input_ids = input_ids + ([0] * padding_length)
    attention_mask = attention_mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
    
    e1_mask = [0] * len(attention_mask)
    e2_mask = [0] * len(attention_mask)
    p1_mask = [0] * len(attention_mask)
    p2_mask = [0] * len(attention_mask)
    if e1 == [] or e2 == []:
        return None
    for index in e1:
        e1_mask[index+1] = 1
        e1_mask[index+2] = 1
        p1_mask[index] = 1
        p1_mask[index+3] = 1
    for index in e2:
        e2_mask[index+1] = 1
        e2_mask[index+2] = 1
        p2_mask[index] = 1
        p2_mask[index+3] = 1
    output['input_ids'] = input_ids
    output['attention_mask'] = attention_mask
    output['token_type_ids'] = token_type_ids
    output['e1_mask'] = e1_mask
    output['e2_mask'] = e2_mask
    output['p1_mask'] = p1_mask
    output['p2_mask'] = p2_mask
    return output

In [ ]:
def load_and_preprocess_data(tokenizer, mode):
    if mode == 'train':
        path = './drive/My Drive/DDI/DDIextraction_2013/train_dataset/'
    if mode == 'test':
        path = './drive/My Drive/DDI/DDIextraction_2013/test_dataset/'
    path_name = ['DrugBank/', 'MedLine/']
    d_list = []
    for name in path_name:
        path_1 = path + name
        for filename in os.listdir(path_1):
            newpath = path_1+filename
            tree = ET.parse(newpath)
            root = tree.getroot()
            for child in root:
            # child is now each sentence
                text = child.attrib['text']
                entity = dict()
                drug_list = []
                for subchild in child:
                  # subchild is now each entity
                  if subchild.tag == 'entity':
                    drug_name = subchild.attrib['text'].lower()
                    entity[subchild.attrib['id']] = drug_name
                    if drug_name not in drug_list:
                      drug_list.append(drug_name)
                for subchild in child:
                  if subchild.tag == 'pair':
                    newtext = text.lower()
                    drug1 = entity[subchild.attrib['e1']]
                    newtext = newtext.replace(drug1, '#drug1#')
                    drug2 = entity[subchild.attrib['e2']]
                    newtext = newtext.replace(drug2, '$drug2$')
                    for drug in drug_list:
                      if drug != drug1 and drug != drug2:
                        newtext = newtext.replace(drug, 'drug0')
                    if drug1 == drug2: #skip data if two same drugs.
                      continue
                    output = help_mask(newtext, tokenizer, max_length = 132)
                    if output is not None:
                      dditype = subchild.attrib['ddi']
                      output['label'] = 0 if dditype == 'false' else 1
                      d_list.append(output)
    # Convert list into tensors.
    all_input_ids = torch.tensor([f['input_ids'] for f in d_list], dtype=torch.long)
    all_attention_mask = torch.tensor([f['attention_mask'] for f in d_list], dtype=torch.long)
    all_token_type_ids = torch.tensor([f['token_type_ids'] for f in d_list], dtype=torch.long)
    all_e1_mask = torch.tensor([f['e1_mask'] for f in d_list], dtype=torch.long)  # add e1 mask
    all_e2_mask = torch.tensor([f['e2_mask'] for f in d_list], dtype=torch.long)  # add e2 mask
    all_p1_mask = torch.tensor([f['p1_mask'] for f in d_list], dtype=torch.long)  # add p1 mask
    all_p2_mask = torch.tensor([f['p2_mask'] for f in d_list], dtype=torch.long)  # add p2 mask
    all_label = torch.tensor([f['label'] for f in d_list], dtype = torch.long)
    dataset = TensorDataset(all_input_ids, all_attention_mask,
                            all_token_type_ids, all_label, all_e1_mask, all_p1_mask, all_e2_mask, all_p2_mask)
    return dataset

In [ ]:
Test_data = load_and_preprocess_data(tokenizer, 'test')
Train_data = load_and_preprocess_data(tokenizer, 'train')
# check data
print(len(Test_data), len(Train_data))

5259 22935


# Build Model

In [ ]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0., use_activation=True):
        super(FCLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.dropout(x)
        if self.use_activation:
            x = self.tanh(x)
        return self.linear(x)

In [ ]:
PRETRAINED_MODEL_MAP = {
    'bert': BertModel
}
class RBERT(BertPreTrainedModel):
    def __init__(self, config, args):
        super(RBERT, self).__init__(config)
        self.bert = PRETRAINED_MODEL_MAP[args.model_type](config=config)  # Load pretrained bert

        self.num_labels = config.num_labels

        self.cls_fc_layer = FCLayer(config.hidden_size, config.hidden_size, args.dropout_rate)
        self.e1_fc_layer = FCLayer(config.hidden_size, config.hidden_size, args.dropout_rate)
        self.e2_fc_layer = FCLayer(config.hidden_size, config.hidden_size, args.dropout_rate)
        self.p1_fc_layer = FCLayer(config.hidden_size, config.hidden_size, args.dropout_rate)
        self.p2_fc_layer = FCLayer(config.hidden_size, config.hidden_size, args.dropout_rate)
        self.label_classifier = FCLayer(config.hidden_size * 5, config.num_labels, args.dropout_rate, use_activation=False)

    @staticmethod
    def entity_average(hidden_output, e_mask):
        """
        Average the entity hidden state vectors (H_i ~ H_j)
        :param hidden_output: [batch_size, j-i+1, dim]
        :param e_mask: [batch_size, max_seq_len]
                e.g. e_mask[0] == [0, 0, 0, 1, 1, 1, 0, 0, ... 0]
        :return: [batch_size, dim]
        """
        e_mask_unsqueeze = e_mask.unsqueeze(1)  # [b, 1, j-i+1]
        length_tensor = (e_mask != 0).sum(dim=1).unsqueeze(1)  # [batch_size, 1]

        sum_vector = torch.bmm(e_mask_unsqueeze.float(), hidden_output).squeeze(1)  # [b, 1, j-i+1] * [b, j-i+1, dim] = [b, 1, dim] -> [b, dim]
        avg_vector = sum_vector.float() / length_tensor.float()  # broadcasting
        return avg_vector

    def forward(self, input_ids, attention_mask, token_type_ids, labels, e1_mask, p1_mask, e2_mask, p2_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids)  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[1]  # [CLS]

        # Average
        e1_h = self.entity_average(sequence_output, e1_mask)
        e2_h = self.entity_average(sequence_output, e2_mask)
        p1_h = self.entity_average(sequence_output, p1_mask)
        p2_h = self.entity_average(sequence_output, p2_mask)

        # Dropout -> tanh -> fc_layer
        pooled_output = self.cls_fc_layer(pooled_output)
        e1_h = self.e1_fc_layer(e1_h)
        e2_h = self.e2_fc_layer(e2_h)
        p1_h = self.p1_fc_layer(p1_h)
        p2_h = self.p2_fc_layer(p2_h)

        # Concat -> fc_layer
        concat_h = torch.cat([pooled_output, e1_h, p1_h, e2_h, p2_h], dim=-1)
        logits = self.label_classifier(concat_h)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        # Softmax
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss(weight= torch.FloatTensor([1., 10.]).cuda())
                #loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
config = BertConfig.from_pretrained('./biobert_v1.1_pubmed_pytorch_model', num_labels = 2)

# Implement Train

In [ ]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    acc = (preds == labels).mean()
    length  = len(preds)
    tp, fp, tn, fn = 0, 0, 0, 0
    for i in range(length):
        if labels[i] == 1:
            if preds[i] == 1:
                tp += 1
            else:
                fn += 1
        else:
            if preds[i] == 1:
                fp += 1
            else:
                tn += 1
    f1 = 2*tp/(2*tp+fp+fn)
    print(tp)
    print(fn)
    print(fp)
    print(tn)
    print(f1)
    return {
        "acc": acc,
        "f1": f1
    }


In [ ]:
MODEL_CLASSES = {
    'bert': (BertConfig, RBERT, BertTokenizer)
}

class Trainer(object):
    def __init__(self, args, train_dataset=None, dev_dataset=None, test_dataset=None):
        self.args = args
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset

        self.num_labels = 2

        self.config_class, self.model_class, _ = MODEL_CLASSES[args.model_type]
        self.config = self.config_class.from_pretrained(args.model_name_or_path, num_labels=self.num_labels, finetuning_task=args.task)
        # finetuning_task should be remove???
        self.model = self.model_class.from_pretrained(args.model_name_or_path,
                                                      config=self.config,
                                                      args=args)

        # GPU or CPU
        self.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
        self.model.to(self.device)

    def train(self):
        # train_sampler = RandomSampler(self.train_dataset) 
        train_sampler = SequentialSampler(self.train_dataset)
        train_dataloader = DataLoader(self.train_dataset, sampler=train_sampler, batch_size=self.args.train_batch_size, drop_last=True)

        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.args.num_train_epochs = self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
        else:
            t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs
        
        
        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': self.args.weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=t_total)

        global_step = 0
        tr_loss = 0.0
        self.model.zero_grad()

        train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch")
        set_seed(self.args)

        for _ in train_iterator:
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                self.model.train()
                batch = tuple(t.to(self.device) for t in batch)  # GPU or CPU
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'token_type_ids': batch[2],
                          'labels': batch[3],
                          'e1_mask': batch[4],
                          'p1_mask': batch[5],
                          'e2_mask': batch[6],
                          'p2_mask': batch[7]}
                outputs = self.model(**inputs)
                loss = outputs[0]
                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                loss.backward()

                tr_loss += loss.item()
                if (step + 1) % self.args.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()
                    global_step += 1

                    if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
                        self.save_model()

                if 0 < self.args.max_steps < global_step:
                    epoch_iterator.close()
                    break
            eval_result = self.evaluate('test')

            if 0 < self.args.max_steps < global_step:
                train_iterator.close()
                break
            print('train loss:')
            print(tr_loss / global_step, eval_result)
        return global_step, tr_loss / global_step, eval_result

    def evaluate(self, mode):
        # We use test dataset because semeval doesn't have dev dataset
        if mode == 'test':
            dataset = self.test_dataset
        elif mode == 'dev':
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None

        self.model.eval()

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'token_type_ids': batch[2],
                          'labels': batch[3],
                          'e1_mask': batch[4],
                          'p1_mask': batch[5],
                          'e2_mask': batch[6],
                          'p2_mask': batch[7]}
                outputs = self.model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs['labels'].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(
                    out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        results = {
            "loss": eval_loss
        }
        preds = np.argmax(preds, axis=1)
        #write_prediction(self.args, os.path.join(self.args.eval_dir, "proposed_answers.txt"), preds)
        result = compute_metrics(preds, out_label_ids)
        results.update(result)
        print('eval results:')
        print(results)
        return results

    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.args.model_dir):
            os.makedirs(self.args.model_dir)
        model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        model_to_save.save_pretrained(self.args.model_dir)

        # Save training arguments together with the trained model
        torch.save(self.args, os.path.join(self.args.model_dir, 'training_args.bin'))
        

    def load_model(self):
        # Check whether model exists
        if not os.path.exists(self.args.model_dir):
            raise Exception("Model doesn't exists! Train first!")

        try:
            self.model = self.model_class.from_pretrained(self.args.model_dir)
            self.model.to(self.device)
            
        except:
            raise Exception("Some model files might be missing...")

In [ ]:
args = easydict.EasyDict({"learning_rate": 2e-5,
        "model_type": 'bert',
        "dropout_rate": 0.01,
        "model_name_or_path": './drive/My Drive/DDI/ep_model',
        # if load saved model use './drive/My Drive/DDI/ep_model'
        # if use bert : 'bert-base-uncased'
        # if load biobert: './biobert_v1.1_pubmed_pytorch_model'
        "task": 'DDI',
        "seed": 42,
        "train_batch_size": 64, # check CUDA memeory, if not large enough change it to 32.
        "eval_batch_size": 64,
        "num_train_epochs": 8,
        "weight_decay": 0,
        "gradient_accumulation_steps": 1,
        "adam_epsilon": 1e-8,
        "max_grad_norm": 1,
        "max_steps": -1,
        "warmup_steps": 0,
        "save_steps": 700,
        "do_train": True,
        "do_eval": True,
        "no_cuda": False,
        "model_dir": "./model"
        })

In [ ]:
trainer = Trainer(args, train_dataset=Train_data, test_dataset=Test_data)

In [ ]:
trainer.train()

Iteration: 100%|██████████| 358/358 [09:31<00:00,  1.60s/it]

Iteration:   0%|          | 0/358 [00:00<?, ?it/s]

872
101
438
3848
0.7639071397284275
eval results:
{'loss': 0.28771072131278647, 'acc': 0.8975090321353869, 'f1': 0.7639071397284275}
train loss:
0.1493131226824867 {'loss': 0.28771072131278647, 'acc': 0.8975090321353869, 'f1': 0.7639071397284275}



Iteration: 100%|██████████| 358/358 [09:49<00:00,  1.65s/it]

Iteration:   0%|          | 0/358 [00:00<?, ?it/s]

817
156
250
4036
0.8009803921568628
eval results:
{'loss': 0.4048696252887687, 'acc': 0.9227990112188629, 'f1': 0.8009803921568628}
train loss:
0.14284533643334135 {'loss': 0.4048696252887687, 'acc': 0.9227990112188629, 'f1': 0.8009803921568628}



Iteration: 100%|██████████| 358/358 [09:47<00:00,  1.64s/it]

Iteration:   0%|          | 0/358 [00:00<?, ?it/s]

...


Iteration: 100%|██████████| 358/358 [09:49<00:00,  1.65s/it]

Iteration:   0%|          | 0/358 [00:00<?, ?it/s]

783
190
183
4103
0.8076328004125838
eval results:
{'loss': 0.5927011524439307, 'acc': 0.9290739684350637, 'f1': 0.8076328004125838}
train loss:
0.10174401100793616 {'loss': 0.5927011524439307, 'acc': 0.9290739684350637, 'f1': 0.8076328004125838}



Iteration: 100%|██████████| 358/358 [09:47<00:00,  1.64s/it]

Iteration:   0%|          | 0/358 [00:00<?, ?it/s]

848
125
244
4042
0.8213075060532687
eval results:
{'loss': 0.472851471067266, 'acc': 0.9298345693097547, 'f1': 0.8213075060532687}
train loss:
0.09499539050100507 {'loss': 0.472851471067266, 'acc': 0.9298345693097547, 'f1': 0.8213075060532687}



Iteration:  16%|█▌        | 58/358 [01:34<08:05,  1.62s/it]

In [ ]:
trainer.save_model()

# load Saved model

In [ ]:
args = easydict.EasyDict({"learning_rate": 2e-5,
        "model_type": 'bert',
        "dropout_rate": 0.01,
        "model_name_or_path": './model',
        "task": 'DDI',
        "seed": 42,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "num_train_epochs": 2,
        "weight_decay": 0,
        "gradient_accumulation_steps": 1,
        "adam_epsilon": 1e-8,
        "max_grad_norm": 1,
        "max_steps": -1,
        "warmup_steps": 0,
        "save_steps": 500,
        "do_train": True,
        "do_eval": True,
        "no_cuda": False,
        "model_dir": "./model"
        })

config = BertConfig.from_pretrained(args.model_name_or_path, num_labels=2)

In [ ]:
trainer = Trainer(args, train_dataset=Train_data, test_dataset=Test_data)

In [ ]:
trainer.evaluate(mode = 'test')

If this model outperforms the previous model, save this model

In [ ]:
!cp -r ./model ./drive/My\ Drive/DDI/ep_model